<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/nlp/RNN_Indian_Name_Generator_Gender_Specific.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Generate Indian Names Randomly via Character RNNs**


Purpose of this NN is to use character RNNs to be able to generate randome names.

The NN will train on indian names data from kaggle.

Network is designed to predict the next character based a sequence of previous characters


In [1]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json
# Next, install the Kaggle API client.
!pip install -q kaggle
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 66 Nov 27 05:12 kaggle.json


In [2]:
!kaggle datasets download -d chaitanyapatil7/indian-names

  0% 0.00/131k [00:00<?, ?B/s]
100% 131k/131k [00:00<00:00, 39.9MB/s]


In [3]:
!unzip indian-names.zip

Archive:  indian-names.zip
  inflating: Indian-Female-Names.csv  
  inflating: Indian-Male-Names.csv   


Till now we have downloaded and setup our data

In [4]:
import torch
from torch import nn

import torch.nn.functional as F

import numpy as np

from sklearn.metrics import accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"

def findFiles(path): return glob.glob(path)
# Read a file and split into lines

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    lines = [line.split(",")[0].strip() for line in lines]
    return [unicodeToAscii(line) for line in lines]

# Build the category_lines dictionary, a list of lines per category

lines1 = readLines("Indian-Female-Names.csv")
lines2 = readLines("Indian-Male-Names.csv")

print("female names", len(lines1))
print("male names", len(lines2))


lines = lines1 + lines2

female names 15383
male names 14846


Read the names from the csv files 

In [6]:
data = []
for l in lines:
   data += list(l)

chars = list(set(data))

m_char = "<"
f_char = ">"


cats_dict = {
    m_char : lines1,
    f_char : lines2
}

chars.append(m_char)
chars.append(f_char)

data_size, vocab_size = len(data), len(chars)

print('data has %d characters, %d unique.' % (data_size, vocab_size))

n_chars = len(chars)

print(chars)

data has 274545 characters, 31 unique.
['l', 'n', 'v', 'm', '-', 'a', 'u', 'c', 'b', 'w', 'g', 'h', 'o', 'f', 'i', 'p', 'k', 'x', 't', ' ', 's', 'z', 'j', 'q', '.', 'd', 'r', 'y', 'e', '<', '>']


In [7]:
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}
print('char_to_ix', char_to_ix)
print('ix_to_char', ix_to_char)

char_to_ix {'l': 0, 'n': 1, 'v': 2, 'm': 3, '-': 4, 'a': 5, 'u': 6, 'c': 7, 'b': 8, 'w': 9, 'g': 10, 'h': 11, 'o': 12, 'f': 13, 'i': 14, 'p': 15, 'k': 16, 'x': 17, 't': 18, ' ': 19, 's': 20, 'z': 21, 'j': 22, 'q': 23, '.': 24, 'd': 25, 'r': 26, 'y': 27, 'e': 28, '<': 29, '>': 30}
ix_to_char {0: 'l', 1: 'n', 2: 'v', 3: 'm', 4: '-', 5: 'a', 6: 'u', 7: 'c', 8: 'b', 9: 'w', 10: 'g', 11: 'h', 12: 'o', 13: 'f', 14: 'i', 15: 'p', 16: 'k', 17: 'x', 18: 't', 19: ' ', 20: 's', 21: 'z', 22: 'j', 23: 'q', 24: '.', 25: 'd', 26: 'r', 27: 'y', 28: 'e', 29: '<', 30: '>'}


In [0]:
# cats = ["male", "female"]

# n_cats = len(cats)

# def one_hot_encode_cat(cat, n_cats):
#   tensor = torch.zeros(n_cats)
#   tensor[cats.index(cat)] = 1
#   return tensor

# print(one_hot_encode_cat("male", n_cats))
# print(one_hot_encode_cat("female", n_cats))

In [0]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, max_seq_length):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   

        # https://discuss.pytorch.org/t/could-someone-explain-batch-first-true-in-lstm/15402

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * max_seq_length, output_size)

        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        # print(out.shape, 'rnn output snape')
        # Reshaping the outputs such that it can be fit into the fully connected layer
        # basically we are concating all the letters together. 
        # the size would be hidden_dim * max_seq_length
        # since our purpose to predect the next character i.e a single character 
        # our network should output only a single character
        out = out.contiguous().view(batch_size, -1)
        # print(out.shape, " new out shape")
        
        
        out = self.dropout(out)
        out = self.fc(out)

        # print(out.shape, 'single layer output shape')
        
        
        out = self.softmax(out)

        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [0]:
max_seq_length = 20
# this will maximum length of a name
# this is required to define the network

model = Model(input_size=n_chars, output_size=n_chars, hidden_dim=12, n_layers=1, max_seq_length=max_seq_length)

# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 1000
lr=0.001

# Define Loss, Optimizer
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [0]:
def one_hot_encode_batch(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape

    # we need to have all seq of same length or else cannot create this array
    # if we don't have batch_size i.e if we process one input at a time. we 
    # don't need to have sequence of same length

    tensor = torch.zeros(batch_size, seq_len, dict_size)

    # len(sequence) and seq_len will be same in case when we are standaring the lenght!

    # features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for li in range(len(sequence[i])):
          tensor[i][li][sequence[i][li]] = 1
    return tensor

In [0]:
# general utility function but not used as such
def one_hot_encode(line, dict_size):
    tensor = torch.zeros(len(line), dict_size)
    for li in range(len(line)):
        tensor[li][line[li]] = 1
    return tensor

In [0]:
def targetTensor(seq):
  tensor = torch.zeros(len(seq))
  for i in range(len(seq)):
    line = seq[i]
    tensor[i] = line
  return tensor.long()

Target tensor is a simple tensor of indexes.
The reason for this because we are use NN Loss, and for we need to provide indexes of character. Its basically is a classification problem with characters a label. 

So our network will output software of possible charaters and we need to compare that with index's of the actual character 

This is important to understand

In [14]:
def train(epoch = -1):
  tloss = 0
  itrloss = 0

  for key, lines in cats_dict.items():
    for line_no in range(len(lines)):
      name = lines[line_no]
      name = name.strip()

      if len(name) <= 1:
        continue

      input_seq_idx = []
      target_char_idx = []
      if len(name) >= max_seq_length: 
        # print(name , " bigger than max seq length FYI")
        name = name[0:max_seq_length - 1]

      name = key + name
      
      for i in range(1, len(name)):
        # input_seq = name[0:i] if i != 0 else start_char
        input_seq = name[0:i]
        target_char = name[i]

        # input_seq = input_seq + end_char
        input_seq = input_seq

        # print(input_seq," ==== ", target_char)

        input_seq_idx.append([char_to_ix[ch] for ch in input_seq])
        target_char_idx.append(char_to_ix[target_char])

      input_encoded = one_hot_encode_batch(input_seq_idx, n_chars, max_seq_length, len(input_seq_idx))
      target_encoded = targetTensor(target_char_idx)

      input_encoded = input_encoded.to(device)
      target_encoded = target_encoded.to(device)
      # print(input_encoded.shape, "input shape")
      

      optimizer.zero_grad()
      output, hidden = model(input_encoded)
      output = output.to(device)
      # output = output

      # print(output.shape, 'final nn ouput shape')
      # print(target_encoded.shape, "target shape")
      # print(target_encoded, "target")

      # The input is expected to contain scores for each class.
      # input has to be a 2D Tensor of size (minibatch, C).
      # This criterion expects a class index (0 to C-1) as the target for each value of a 1D tensor of size minibatch

      loss = criterion(output, target_encoded)
      loss.backward() # Does backpropagation and calculates gradients
      optimizer.step() # Updates the weights accordingly
      tloss += loss 
      itrloss += loss

      

      if line_no%1000 == 0 and epoch == -1:
        print("Iteration Loss: {}/{}...".format(line_no, itrloss/1000))
        prediction = output.argmax(dim=1)
        itrloss = 0

  print("Epoch: {}/ Loss: {}".format(epoch, tloss / len(lines)))
  return tloss

train()




Iteration Loss: 0/0.003416462102904916...
Iteration Loss: 1000/2.821967840194702...
Iteration Loss: 2000/2.6630442142486572...
Iteration Loss: 3000/2.5244359970092773...
Iteration Loss: 4000/2.3408658504486084...
Iteration Loss: 5000/2.314293384552002...
Iteration Loss: 6000/2.2525391578674316...
Iteration Loss: 7000/2.238591432571411...
Iteration Loss: 8000/2.2567570209503174...
Iteration Loss: 9000/2.227295160293579...
Iteration Loss: 10000/2.220383644104004...
Iteration Loss: 11000/2.1699678897857666...
Iteration Loss: 12000/2.1516005992889404...
Iteration Loss: 13000/2.156041383743286...
Iteration Loss: 14000/2.1162173748016357...
Iteration Loss: 15000/2.1084890365600586...
Iteration Loss: 0/0.8259483575820923...
Iteration Loss: 1000/2.519371271133423...
Iteration Loss: 2000/2.330448865890503...
Iteration Loss: 3000/2.318242073059082...
Iteration Loss: 4000/2.2764854431152344...
Iteration Loss: 5000/2.225952625274658...
Iteration Loss: 6000/2.2224717140197754...
Iteration Loss: 700

tensor(68533.8906, device='cuda:0', grad_fn=<AddBackward0>)

In [15]:
import random

def eval():
  x = random.randrange(100,500)
  # print("index" , x)
  eval_lines = lines[x:x+200]

  input_seq_idx = []
  target_char_idx = []
  for line_no in range(len(eval_lines)):
      name = lines[line_no]
      name = name.strip()

      if len(name) == 0:
        continue

      if len(name) >= max_seq_length: 
        name = name[0:max_seq_length - 1]
      
      for i in range(1, len(name)):
        input_seq = name[0:i]
        target_char = name[i]

        # input_seq = input_seq + end_char
        input_seq = input_seq

        input_seq_idx.append([char_to_ix[ch] for ch in input_seq])
        target_char_idx.append(char_to_ix[target_char])

  input_encoded = one_hot_encode_batch(input_seq_idx, n_chars, max_seq_length, len(input_seq_idx))
  target_encoded = targetTensor(target_char_idx)

  input_encoded = input_encoded.to(device)
  target_encoded = target_encoded.to(device)

  # print(input_encoded.shape, "input shape")
  # print(target_encoded.shape, "target shape")

  with torch.no_grad():

    output, _ = model(input_encoded)

    output = output.to(device)

    prediction = output.argmax(dim=1).cpu()
    acc = accuracy_score(target_encoded.cpu(), prediction)
    print("Accuracy {}".format(acc))


eval()

Accuracy 0.1866096866096866


In [16]:
epochs = 20
for epoch in range(epochs):
  train(epoch)
  # eval()

Epoch: 0/ Loss: 4.286904811859131
Epoch: 1/ Loss: 4.184067249298096
Epoch: 2/ Loss: 4.130712032318115
Epoch: 3/ Loss: 4.097538471221924
Epoch: 4/ Loss: 4.073625564575195
Epoch: 5/ Loss: 4.057404518127441
Epoch: 6/ Loss: 4.044101715087891
Epoch: 7/ Loss: 4.034395694732666
Epoch: 8/ Loss: 4.0260491371154785
Epoch: 9/ Loss: 4.017147541046143
Epoch: 10/ Loss: 4.007729530334473
Epoch: 11/ Loss: 4.00342321395874
Epoch: 12/ Loss: 3.9968605041503906
Epoch: 13/ Loss: 3.9957475662231445
Epoch: 14/ Loss: 3.9902591705322266
Epoch: 15/ Loss: 3.9878318309783936
Epoch: 16/ Loss: 3.984039068222046
Epoch: 17/ Loss: 3.9872138500213623
Epoch: 18/ Loss: 3.9813859462738037
Epoch: 19/ Loss: 3.980846881866455


In [21]:
def generate(input_seq_sample):
  # input_seq_sample = input_seq_sample + end_char
  input_seq_sample = input_seq_sample
  # print(input_seq_sample)
  input_seq_idx = []
  input_seq_idx.append([char_to_ix[ch] for ch in input_seq_sample])
  input_encoded = one_hot_encode_batch(input_seq_idx, n_chars, max_seq_length, len(input_seq_idx))

  input_encoded = input_encoded.to(device)

  output, _ = model(input_encoded)

  prediction = output.argmax(dim=1).cpu().numpy().data

  text = [ix_to_char[idx] for idx in prediction ]
  # print(text)

  return "".join(text)


def generate_name(input_seq_sample, name_length):
  print("generate name with starting seq '{}' and of size '{}'".format(input_seq_sample, name_length))

  for i in range(name_length):
    ret = generate(input_seq_sample)
    # print(input_seq_sample, " :::: ", ret)
    input_seq_sample += ret

  print("Name Generated {}".format(input_seq_sample))

  return input_seq_sample

def run_random_generation(cat):
  init_seq_size = random.randrange(0,2)
  init_seq_index = random.randrange(0, len(chars) - init_seq_size)
  init_seq = chars[init_seq_index: init_seq_index + init_seq_size]

  input_seq_sample = cat + "".join(init_seq)
  name_length = random.randrange(1, max_seq_length-len(input_seq_sample))

  generate_name(input_seq_sample,name_length)

  

for i in range(10):
  run_random_generation(m_char)
  run_random_generation(f_char)

generate_name(m_char, 10)
generate_name(f_char, 10)

generate name with starting seq '<a' and of size '16'
Name Generated <ankit singharingi
generate name with starting seq '>h' and of size '10'
Name Generated >haresh sing
generate name with starting seq '<b' and of size '5'
Name Generated <bhar k
generate name with starting seq '>' and of size '1'
Name Generated >s
generate name with starting seq '<' and of size '14'
Name Generated <sharman singhh
generate name with starting seq '>v' and of size '3'
Name Generated >vino
generate name with starting seq '<s' and of size '11'
Name Generated <shiv kumarar
generate name with starting seq '>' and of size '12'
Name Generated >shand kumarr
generate name with starting seq '<m' and of size '14'
Name Generated <mohonum kumarir
generate name with starting seq '>' and of size '5'
Name Generated >shara
generate name with starting seq '<k' and of size '7'
Name Generated <karender
generate name with starting seq '>c' and of size '6'
Name Generated >chand k
generate name with starting seq '<.' and of si

'>sharam  bh'

">" means Male names

"<" means Female *names*


In [0]:
# criterion = nn.NLLLoss()

# output = torch.randn(10, 120).float()
# target = torch.FloatTensor(10).uniform_(0, 120).long()

# print(output.shape)
# print(target.shape)
# print(target)

# loss = criterion(output, target)